In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

import sys
sys.path.append('../utilities/')
import column_name_mapping as cnm
import general_tools as gt
import dataframe_operations as do

In [2]:
data_raw    = pd.read_csv(Path.cwd() / '..' / 'data' / 'ml_case_training_data.csv')
data_output = pd.read_csv(Path.cwd() / '..' / 'data' / 'ml_case_training_output.csv')

In [5]:
do.pre_merge_report(data_raw, data_output, how='inner', key='id')

,shape,unique keys,keys not in other
DF 1,"(16096, 32)",16096,0
DF 2,"(16096, 2)",16096,0
Merged,"(16096, 33)",,
